In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.models import Sequential
import importlib
import make_data as md
importlib.reload(md)
import make_model as mm
importlib.reload(mm)
import optuna
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
from datetime import datetime as dt
from datetime import timedelta

In [19]:
data, data_np = md.load_data("日経平均00-22日足.csv")

In [20]:
data["アッパーエンベロープー終値"] = md.upper_envelope(3, 5, 1, data_np)
data["ロワーエンベロープー終値"] = md.lower_envelope(3, 5, 1, data_np)
data["5日平均-25日平均"] = md.average_line_difference(data["5日平均"], data["25日平均"])
data["25日平均-75日平均"] = md.average_line_difference(data["25日平均"], data["75日平均"])
data_5_25 = md.cross(data["5日平均-25日平均"])
data_25_75 = md.cross(data["25日平均-75日平均"])
upper_bband, lower_bband = md.bband(data)
data["アッパーボリンジャーバンド"] = upper_bband
data["ロワーボリンジャーバンド"] = lower_bband
data["ボリンジャーバンド差"] = md.average_line_difference(data["アッパーボリンジャーバンド"] ,data["ロワーボリンジャーバンド"])
data_bband_border = md.bband_border(data["アッパーボリンジャーバンド"], data["ロワーボリンジャーバンド"], data["高値"], data["安値"])
data["MACD"] =  [x - y for (x, y) in zip(md.ema(data["終値"], 9), md.ema(data["終値"], 26))]
data["シグナルMACD"] = md.average_line(data["MACD"],9)
data_macdcross = md.cross(md.average_line_difference(data["MACD"], data["シグナルMACD"]))
data_up_or_down = md.up_or_down(data["終値"])
data_close=data["終値"]

In [21]:
data = md.standardization(data)

In [22]:
data_days, data_weeks = md.data_from_year(data.index)
#data["年始からの日"] = data_days
#data["年始からの週"] = data_weeks
data['5-25クロス'] = data_5_25
data['25-75クロス'] = data_25_75
data["ボリンジャーバンドボーダー"] = data_bband_border
data["rsi9"] = md.rsi(data["終値"],9)
data["rsi14"] = md.rsi(data["終値"],14)
data["MACDクロス"] = data_macdcross

In [23]:
data = data.drop(["高値","安値","5日平均.1","25日平均.1", '5日平均', '25日平均', '75日平均','アッパーボリンジャーバンド', 'ロワーボリンジャーバンド'], axis=1)

In [24]:
data["騰落"]=data_up_or_down
data = data.dropna()
data_np = data.to_numpy()

In [26]:
seq_len = 30
epoch = 500
x_train, y_train, x_test, y_test= mm.data_split(data_np,seq_len=seq_len, y=-1)
model = mm.build_model([data.shape[1],50,100,1])

In [27]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.fit(x_train, y_train, batch_size=512, epochs=epoch, validation_split=0.05, callbacks=[callback])

Epoch 1/500
8/8 [==============================] - 5s 645ms/step - loss: 0.3400 - mae: 0.5197 - acc: 0.4865 - val_loss: 0.2650 - val_mae: 0.4946 - val_acc: 0.5070
Epoch 2/500
8/8 [==============================] - 3s 429ms/step - loss: 0.2711 - mae: 0.5040 - acc: 0.4824 - val_loss: 0.2584 - val_mae: 0.4996 - val_acc: 0.4605
Epoch 3/500
8/8 [==============================] - 3s 429ms/step - loss: 0.2622 - mae: 0.5003 - acc: 0.4939 - val_loss: 0.2522 - val_mae: 0.4960 - val_acc: 0.5442
Epoch 4/500
8/8 [==============================] - 3s 422ms/step - loss: 0.2557 - mae: 0.4962 - acc: 0.5172 - val_loss: 0.2529 - val_mae: 0.5008 - val_acc: 0.5163
Epoch 5/500
8/8 [==============================] - 3s 420ms/step - loss: 0.2539 - mae: 0.4965 - acc: 0.5157 - val_loss: 0.2490 - val_mae: 0.4940 - val_acc: 0.5488
Epoch 6/500
8/8 [==============================] - 3s 424ms/step - loss: 0.2522 - mae: 0.4940 - acc: 0.5181 - val_loss: 0.2480 - val_mae: 0.4954 - val_acc: 0.5349
Epoch 7/500
8/8 [=====

Epoch 47/500
8/8 [==============================] - 3s 436ms/step - loss: 0.1779 - mae: 0.3691 - acc: 0.7277 - val_loss: 0.1821 - val_mae: 0.3664 - val_acc: 0.7395
Epoch 48/500
8/8 [==============================] - 4s 447ms/step - loss: 0.1767 - mae: 0.3646 - acc: 0.7358 - val_loss: 0.1810 - val_mae: 0.3652 - val_acc: 0.7442
Epoch 49/500
8/8 [==============================] - 3s 432ms/step - loss: 0.1767 - mae: 0.3654 - acc: 0.7338 - val_loss: 0.1818 - val_mae: 0.3651 - val_acc: 0.7488
Epoch 50/500
8/8 [==============================] - 4s 456ms/step - loss: 0.1766 - mae: 0.3631 - acc: 0.7365 - val_loss: 0.1820 - val_mae: 0.3656 - val_acc: 0.7395
Epoch 51/500
8/8 [==============================] - 4s 484ms/step - loss: 0.1799 - mae: 0.3657 - acc: 0.7301 - val_loss: 0.1846 - val_mae: 0.3678 - val_acc: 0.7256
Epoch 52/500
8/8 [==============================] - 4s 459ms/step - loss: 0.1733 - mae: 0.3622 - acc: 0.7456 - val_loss: 0.1817 - val_mae: 0.3716 - val_acc: 0.7535
Epoch 53/500
8/8

In [28]:
mm.evalution(x_test,y_test,model)

混同行列[[160  76]
 [ 65 176]]
正解率は0.7044025157232704 適合率は0.7111111111111111 再現率は0.6779661016949152 F値は0.6941431670281994


In [29]:
x_test_close=data_close[(len(data_close)-len(x_test))-1:-1]
value = mm.baibai(x_test,model,x_test_close)
print(value)

16965.75999999998


In [49]:
model.save("my_model2")
#reconstructed_model = tf.keras.models.load_model("my_model")
#mm.draw_tran_result(reconstructed_model, x_test, y_test)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model2\assets


In [16]:
x_train_sfs=data.iloc[:,:-1]
y_train_sfs=data.iloc[:,-1]

sfs1 = SFS(RandomForestClassifier(), # 使う学習器
           k_features=10, #特徴をいくつまで選択するか
           forward=True, #Trueでforward selectionになる。Falseでback
           floating=False, #後に説明するが、forward selectionの亜種を行うためのもの。
           verbose=2, #実行時のlogをどれだけ詳しく表示するか
           scoring='accuracy', # 評価指標
           cv=5) #クロスバリデーション
#ちなみにクロスバリデーションをしない(cv=0)とすると、最終的に選ばれる特徴が変わります。

sfs1 = sfs1.fit(x_train_sfs,y_train_sfs)

selected_feat = x_train_sfs.columns[list(sfs1.k_feature_idx_)]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.1min finished

[2022-06-13 17:19:19] Features: 1/10 -- score: 0.5242680540924043[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:   50.9s finished

[2022-06-13 17:20:10] Features: 2/10 -- score: 0.5105736030787091[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:   48.1s finished

[2022-06-13 17:20:58] Features: 3/10 -- score: 0.5157654849181948[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [17]:
sfs1.subsets_

{1: {'feature_idx': (11,),
  'cv_scores': array([0.52227979, 0.50569948, 0.53008299, 0.53630705, 0.52697095]),
  'avg_score': 0.5242680540924043,
  'feature_names': ('rsi8',)},
 2: {'feature_idx': (11, 13),
  'cv_scores': array([0.51606218, 0.51917098, 0.51763485, 0.49273859, 0.50726141]),
  'avg_score': 0.5105736030787091,
  'feature_names': ('rsi8', 'rsi16')},
 3: {'feature_idx': (7, 11, 13),
  'cv_scores': array([0.51088083, 0.49948187, 0.52385892, 0.52593361, 0.5186722 ]),
  'avg_score': 0.5157654849181948,
  'feature_names': ('出来高', 'rsi8', 'rsi16')},
 4: {'feature_idx': (7, 11, 13, 14),
  'cv_scores': array([0.5357513 , 0.50880829, 0.53215768, 0.52489627, 0.51037344]),
  'avg_score': 0.5223973942768688,
  'feature_names': ('出来高', 'rsi8', 'rsi16', 'rsi20')},
 5: {'feature_idx': (7, 9, 11, 13, 14),
  'cv_scores': array([0.53367876, 0.51709845, 0.52697095, 0.51763485, 0.51348548]),
  'avg_score': 0.5217736976759185,
  'feature_names': ('出来高', '25日平均.1', 'rsi8', 'rsi16', 'rsi20')},
 